# OP2 Demo - Numpy

The Jupyter notebook for this demo can be found in:
   - docs\quick_start\demo\op2_demo.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_demo.ipynb

It's recommended that you first go through:
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_intro.ipynb


The previous demo was intentionally clunky to demonstrate how one might think of a single element. 

If you code like that, your code will be slow, so let's show you how to really use the numpy-style with the OP2.

## Import the packages

In [1]:
import os
import copy
import numpy as np
np.set_printoptions(precision=2, threshold=20, suppress=True)

import pyNastran
pkg_path = pyNastran.__path__[0]
model_path = os.path.join(pkg_path, '..', 'models')

from pyNastran.utils import print_bad_path
from pyNastran.op2.op2 import read_op2
from pyNastran.utils import object_methods, object_attributes
np.set_printoptions(precision=3, threshold=20, edgeitems=10)

### Load the model

#### TODO: consider changing the model to something static with a CD frame and more element types (e.g., CTETRA/CQUAD4/CTRIA3).

In [5]:
from pyNastran.op2.op2 import read_op2

op2_filename = os.path.join(model_path, 'solid_bending', 'solid_bending.op2')
model = read_op2(op2_filename, debug=False)

INFO:    op2_scalar.py:1306           op2_filename = 'c:\\nasa\\m4\\formats\\git\\pynastran\\pyNastran\\..\\models\\solid_bending\\solid_bending.op2'


### Find the min/max displacement magnitude

In [6]:
subcase_id = 1
disp = model.displacements[subcase_id]
print('headers = %s' % (disp.get_headers()))

txyz = disp.data[0, :, :3]
txyz_mag = np.linalg.norm(txyz, axis=1)
txyz_mag_max = txyz_mag.max()
txyz_mag_min = txyz_mag.min()

inid_max = np.where(txyz_mag == txyz_mag_max)[0]
inid_min = np.where(txyz_mag == txyz_mag_min)[0]
all_nodes = disp.node_gridtype[:, 0]
max_nodes = all_nodes[inid_max]
min_nodes = all_nodes[inid_min]
print('max displacement=%s max_nodes=%s' % (txyz_mag_max, max_nodes))
print('min displacement=%s max_nodes=%s' % (txyz_mag_min, min_nodes))

headers = [u't1', u't2', u't3', u'r1', u'r2', u'r3']
max displacement=0.012376265 max_nodes=[23]
min displacement=0.0 max_nodes=[31 35 39 43 47 48 53 63 64 69 70 71 72]


### Find the max centroidal stress on the CTETRA elements

In [7]:
subcase_id = 1
stress = model.ctetra_stress[subcase_id]
print('headers = %s' % (stress.get_headers()))

element_node = stress.element_node
elements = element_node[:, 0]
nodes = element_node[:, 1]
print(element_node)

headers = [u'oxx', u'oyy', u'ozz', u'txy', u'tyz', u'txz', u'omax', u'omid', u'omin', u'von_mises']
[[  1   0]
 [  1   8]
 [  1  13]
 [  1  67]
 [  1  33]
 [  2   0]
 [  2   8]
 [  2   7]
 [  2  62]
 [  2  59]
 ...
 [185   0]
 [185  54]
 [185  39]
 [185  64]
 [185  71]
 [186   0]
 [186   8]
 [186  62]
 [186   4]
 [186  58]]


#### The 0 location is the centroid

You can either query the 0 location or calculate it with a numpy arange.  CTETRA elements have 4 nodes (even 10 noded CTETRA elements) in the OP2.

In [8]:
izero = np.where(nodes == 0)[0]
izero2 = np.arange(0, len(nodes), step=5, dtype='int32')
#print(izero)
#print(izero2)
eids_centroid = elements[izero2]
print('eids_centroid = %s' % eids_centroid)

vm_stress = stress.data[0, izero2, -1]
print(vm_stress)

vm_stress_max = vm_stress.max()
vm_stress_min = vm_stress.min()
icentroid_max = np.where(vm_stress == vm_stress_max)[0]
icentroid_min = np.where(vm_stress == vm_stress_min)[0]
eids_max = eids_centroid[icentroid_max]
eids_min = eids_centroid[icentroid_min]

print('max_stress=%s eids=%s' % (vm_stress_max, eids_max))
print('min_stress=%s eids=%s' % (vm_stress_min, eids_min))

eids_centroid = [  1   2   3   4   5   6   7   8   9  10 ... 177 178 179 180 181 182 183
 184 185 186]
[15900.173 16272.253 12798.722 10728.189 26309.43  30346.639 45438.992
 51427.406 40912.426 41191.414 ...  7342.325 10163.439 28830.463 46618.023
  6998.956  7861.917  8589.076  6053.971 44450.695 22886.705]
max_stress=52446.37 eids=[142]
min_stress=3288.5732 eids=[165]
